<a href="https://colab.research.google.com/github/rajsanshi/Multi_Agent_Customer_Service/blob/main/Mullti_Agent_Customer_Support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **L3: Multi-agent Customer Support Automation¶**

Agents perform:

1.Role Playing
2.Focus
3.Tools
4.Cooperation
5.Guardrails
6.Memory

In [1]:
import warnings
warnings.filterwarnings('ignore')



In [ ]:
pip install crewai

In [3]:
from crewai import Agent,Task,Crew

# **LLM**

Running local model using ollama but using openai method by giving a fake api-key

In [4]:
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-1111"

llm = ChatOpenAI(
    model="ollama/llama3.2",
    base_url="http://localhost:11434/v1"
)



# **Agents**

In [5]:
support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful support representative in your team",
    backstory=(
        "You work at a software consulting firm and are now are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
    ),
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [6]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at software consulting firm and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
  llm=llm,
	verbose=True
)

# **Tools , Guardrails and Memory**

In [7]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [8]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.langchain.com/"
)

# **Creating Tasks**

In [9]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [10]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


# **Creating Crew**

In [11]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

In [ ]:
from IPython.display import Markdown


inputs = {
    "customer": "ACME Inc.",
    "person": "Jane Doe",
    "inquiry": "We are trying to integrate a memory component into our workflow with LangChain. How can we persist and retrieve conversation history?"
}

result = crew.kickoff(inputs=inputs)

Markdown(result)